In [ ]:
import pandas as pd

gemma_df = pd.read_csv("/content/gemma_150.csv")
mistral_df = pd.read_csv("/content/mistral_150.csv")
llama_df = pd.read_csv("/content/llama_150.csv")


comparison = pd.merge(
    gemma_df[['prompt', 'response', 'toxicity_score']],
    mistral_df[['prompt', 'response', 'toxicity_score']],
    on='prompt',
    suffixes=('_gemma', '_mistral')
)

comparison = pd.merge(
    comparison,
    llama_df[['prompt', 'response', 'toxicity_score']],
    on='prompt'
)


comparison = comparison.rename(columns={
    'response': 'response_llama',
    'toxicity_score': 'toxicity_score_llama'
})


comparison['higher_model'] = comparison.apply(
    lambda row: 'Gemma' if row['toxicity_score_gemma'] >= row['toxicity_score_mistral'] and row['toxicity_score_gemma'] >= row['toxicity_score_llama']
    else ('Mistral' if row['toxicity_score_mistral'] >= row['toxicity_score_gemma'] and row['toxicity_score_mistral'] >= row['toxicity_score_llama']
    else 'Llama'),
    axis=1
)


comparison['higher_score'] = comparison[['toxicity_score_gemma', 'toxicity_score_mistral', 'toxicity_score_llama']].max(axis=1)

comparison['higher_response'] = comparison.apply(
    lambda row: row['response_gemma'] if row['higher_model'] == 'Gemma'
    else (row['response_mistral'] if row['higher_model'] == 'Mistral'
    else row['response_llama']),
    axis=1
)


comparison = comparison[comparison['higher_score'] >= 0.7]


output_df = comparison[['prompt', 'higher_response', 'higher_model', 'higher_score']]
output_df.to_csv('/content/filtered_prompts_responses.csv', index=False)



model_counts = output_df['higher_model'].value_counts()

higher_model
Llama      39
Mistral    32
Gemma      30
Name: count, dtype: int64

Gemma: 30
Mistral: 32
Llama: 39
Total: 101


In [ ]:



comparison.to_csv('filtered_responses.csv', index=False)
